<a href="https://colab.research.google.com/github/ljhoh1/Dissertation_Data_2/blob/master/Dissertation_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import py_entitymatching as em
except:
    !pip install py_entitymatching

try:
    import deepmatcher
except:
    !pip uninstall preprocessing -y
    !pip uninstall fastai -y 
    !pip uninstall allennlp -y
    !pip install deepmatcher

import pandas as pd
if pd.__version__ != "1.2.4":
 !pip install pandas==1.2.4
 import pandas as pd
 print(f"Verify pandas=={pd.__version__}")
else:
 print(f"Verify pandas=={pd.__version__}")

import sys
import tensorflow as tf
from tensorflow.python.client import device_lib
import deepmatcher as dm
import py_entitymatching as em
import pandas as pd
import os
from collections import defaultdict
import numpy as np

Verify pandas==1.2.4


In [2]:
! git clone https://github.com/ljhoh1/Dissertation_Data_2

fatal: destination path 'Dissertation_Data_2' already exists and is not an empty directory.


In [3]:
# The path to the two input tables.
path_Scholar = os.path.join('.', 'Dissertation_Data_2', 'Scholar_data.csv')
path_DBLP = os.path.join('.', 'Dissertation_Data_2', 'DBLP1_data.csv')

In [4]:
# checking the character embedding
import chardet
with open(path_Scholar, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}

In [5]:
import chardet
with open(path_DBLP, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'ISO-8859-1', 'confidence': 0.7298232262081848, 'language': ''}

In [6]:
# Load the Scholar data table
Scholar = em.read_csv_metadata(path_Scholar, key='id')
# Transforming the title column cells
Scholar['title'] = Scholar['title'].str.lower()
Scholar['title'] = Scholar['title'].str.replace('[^A-Za-z0-9 ]+', '')
Scholar.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


,id,title,authors,venue,year
0,aKcZKwvwbQwJ,11578 sorrento valley road,QD Inc,"San Diego,",NaN
1,ixKfiTHoaDoJ,initiation of crazes in polystyrene,"AS Argon, JG Hannoosh","Phil. Mag,",NaN
2,3BxllB4wwcIJ,immunogold labelling is a quantitative method as demonstrated by studies on aminopeptidase n in,"GH Hansen, LL Wetterberg, H SjÃ¶strÃ¶m, O NorÃ©n","The Histochemical Journal,",1992.0
3,d2WWxwKMex4J,the burden of infectious disease among inmates and releasees from correctional facilities,"TM Hammett, P Harmon, W Rhodes",see,NaN
4,cZCX-AQpjccJ,the role of faculty advising in science and engineering,JR Cogdell,"NEW DIRECTIONS FOR TEACHING AND LEARNING,",1995.0


In [7]:
DBLP = em.read_csv_metadata(path_DBLP, key='id')
DBLP['title'] = DBLP['title'].str.replace('[^A-Za-z0-9 ]+', '')
DBLP['title'] = DBLP['title'].str.lower()
DBLP.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,id,title,authors,venue,year
0,conf/vldb/RusinkiewiczKTWM95,towards a cooperative transaction model the cooperative activity model,"M Rusinkiewicz, W Klas, T Tesch, J W�sch, P Muth",VLDB,1995
1,journals/sigmod/EisenbergM02,sqlxml is making good progress,"A Eisenberg, J Melton",SIGMOD Record,2002
2,conf/vldb/AmmannJR95,using formal methods to reason about semanticsbased decompositions of transactions,"P Ammann, S Jajodia, I Ray",VLDB,1995
3,journals/sigmod/Liu02,editors notes,L Liu,SIGMOD Record,2002
4,journals/sigmod/Hammer02,report on the acm fourth international workshop on data warehousing and olap dolap 2001,NaN,NaN,2002


In [8]:
# Get tokenizers and similarity function
block_t = em.get_tokenizers_for_blocking()
block_s = em.get_sim_funs_for_blocking()

# Get attributes
atypes1 = em.get_attr_types(Scholar)
atypes2 = em.get_attr_types(DBLP)

# Get correspondence
block_c = em.get_attr_corres(Scholar, DBLP)

#Get Features
block_f = em.get_features(Scholar, DBLP, atypes1, atypes2, block_c, block_t, block_s)

In [9]:
# Creating a rule based blocker
rb = em.RuleBasedBlocker()
rule = ['title_title_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.4']
rb.add_rule(rule, feature_table=block_f)
C = rb.block_tables(Scholar, DBLP,
                   l_output_attrs=['title', 'authors', 'venue', 'year'], 
                   r_output_attrs=['title', 'authors', 'venue', 'year'], 
                   n_jobs=1)


0% [##############################] 100% | ETA: 00:00:00

Finding pairs with missing value...



Total time elapsed: 00:00:42


In [10]:
# The number of tuple pairs in D.
len(C)

11427

In [11]:
C.head()

,_id,ltable_id,rtable_id,ltable_title,ltable_authors,ltable_venue,ltable_year,rtable_title,rtable_authors,rtable_venue,rtable_year
0,0,019zSr3Lx4EJ,conf/vldb/RusinkiewiczKTWM95,towards a cooperative activity modelthe coopertive activity model,"M Rusinkiewicz, W Klas, T Tesch, J Wasch, P Muth","Proceedings of the 21st International Conference on Very &hellip;,",NaN,towards a cooperative transaction model the cooperative activity model,"M Rusinkiewicz, W Klas, T Tesch, J W�sch, P Muth",VLDB,1995
1,1,KureLsKbUXYJ,conf/vldb/RusinkiewiczKTWM95,towaxds a cooperative transaction modelthe cooperative activity model,"M Rnsinkiewicz, W Klasâ?¦","Proc. Int. Conf. Very Large Data Bases, Dayal U, Gray P, &hellip;,",NaN,towards a cooperative transaction model the cooperative activity model,"M Rusinkiewicz, W Klas, T Tesch, J W�sch, P Muth",VLDB,1995
2,2,T2fm7Wb1ak4J,conf/vldb/RusinkiewiczKTWM95,towards a cooperative transaction modelthe cooperative activity model,"M Rusinkiewicz, W Klas, T Tesch, J Waesch, P Muth","PROCEEDINGS OF THE INTERNATIONAL CONFERENCE ON VERY LARGE &hellip;,",1995.0,towards a cooperative transaction model the cooperative activity model,"M Rusinkiewicz, W Klas, T Tesch, J W�sch, P Muth",VLDB,1995
3,3,iWNLOYCQX-YJ,conf/vldb/RusinkiewiczKTWM95,w asch j muth p towards a cooperative transaction modelthe cooperative activity model,"M Rusinkiewicz, W Klas, T Tesch",Proc. ofthe 21 stInternational Conference on Very Large,NaN,towards a cooperative transaction model the cooperative activity model,"M Rusinkiewicz, W Klas, T Tesch, J W�sch, P Muth",VLDB,1995
4,4,wgK6p4mDSIMJ,journals/sigmod/EisenbergM02,sqlxml is making good progress,"A Eisenberg, J Melton","SIGMOD Record,",2002.0,sqlxml is making good progress,"A Eisenberg, J Melton",SIGMOD Record,2002


In [12]:
# The path to the labeled data file.
path_Match = os.path.join('.', 'Dissertation_Data_2', 'DBLP-Scholar_Perfect_Mapping.csv')

In [13]:
# Load the labeled data into a dataframe.
M = em.read_csv_metadata(path_Match, 
                         ltable=Scholar, rtable=DBLP, 
                         fk_ltable='idScholar', fk_rtable='idDBLP')
print('Number of labeled pairs:', len(M))

Number of labeled pairs: 5347


In [14]:
L = em.read_csv_metadata(path_Match)

In [15]:
L.shape

(5347, 2)

In [16]:
L.head()

,idDBLP,idScholar
0,conf/sigmod/AbadiC02,f2Lea-RN8dsJ
1,conf/sigmod/AbadiCCCCEGHMRSSTXYZ03,eBnT7lhV2LwJ
2,conf/sigmod/AbadiCCCCEGHMRSSTXYZ03,gBVNSFeS4P8J
3,conf/sigmod/AbadiCCCCEGHMRSSTXYZ03,VuY9Y49GqXgJ
4,conf/sigmod/AbiteboulBCMM03,AxpQwgyRyLgJ


In [38]:
# create dictionary for checking how many matches are retained and perform labelling
mapping = defaultdict(lambda: [])
for row in C.iterrows():
  mapping[row[1]["rtable_id"]].append(row[1]["ltable_id"])

ct = 0
for row in L.iterrows():
  id_dbl = row[1]["idDBLP"]
  id_sch = row[1]["idScholar"]
  if id_sch in mapping[id_dbl]:
    ct += 1
    C.loc[(C['rtable_id'] == id_dbl) & (C['ltable_id'] == id_sch), 'label'] = 1  
C.loc[(C['label'] != 1), 'label'] = 0

In [29]:
print("Proportion of matches retained after blocking:", round(ct / L.shape[0], 2))

Proportion of matches retained after blocking: 0.99


In [19]:
def grid_search_block(dataL = Scholar, dataR = DBLP, matched = L):
  '''
  Checking the balance between positive and negative matches and the positive 
  matches retained by gridsearch.
  Input:
  - dataL (pandas DataFrame): Data Frame containing the left table
  - dataR (pandas DataFrame): Data Frame containing the right table
  - L (pandas DataFrame): Data Frame containing the actual matches

  Output:
  - results (matrix): matrix containing balance and retained matches information
  '''
  # Get tokenizers and similarity function
  block_t = em.get_tokenizers_for_blocking()
  block_s = em.get_sim_funs_for_blocking()

  # Get attributes
  atypes1 = em.get_attr_types(Scholar)
  atypes2 = em.get_attr_types(DBLP)

  # Get correspondence
  block_c = em.get_attr_corres(Scholar, DBLP)

  #Get Features
  block_f = em.get_features(dataL, dataR, atypes1, atypes2, block_c, block_t, block_s)


  # result data frame
  results = np.zeros((21, 2))
  i = 0
  # grid search
  for thr in np.linspace(0.5, 0.3, 21):
    print(thr)
    # Creating a rule based blocker
    rb = em.RuleBasedBlocker()
    rule = [f'title_title_jac_qgm_3_qgm_3(ltuple, rtuple) < {thr}']
    rb.add_rule(rule, feature_table=block_f)
    l_attr = list(dataL.columns)
    r_attr = list(dataR.columns)
    C = rb.block_tables(dataL, dataR,
                      l_output_attrs=l_attr, 
                        r_output_attrs=r_attr, 
                      n_jobs=1)
    
    # create dictionary for checking how many matches are retained
    mapping = defaultdict(lambda: [])
    for row in C.iterrows():
      mapping[row[1]["rtable_id"]].append(row[1]["ltable_id"])

    ct = 0
    for row in L.iterrows():
      id_dbl = row[1]["idDBLP"]
      id_sch = row[1]["idScholar"]
      if id_sch in mapping[id_dbl]:
        ct += 1

    results[i, 0] = round(ct / L.shape[0], 2)
    results[i, 1] = round(ct / C.shape[0], 2)
    i += 1
  # results.columns = ["PercRet", "PosNegRat"]
  
  return results
    

In [ ]:
grid_search_block()

In [30]:
path_Cand = os.path.join('.', 'Dissertation_Data_2')
em.save_table(C, path_Cand)

True

In [31]:
cand = em.load_table('./Dissertation_Data_2/cand.pkl', metadata_ext='.pklmeta')

In [39]:
sum(C["label"])

5300

In [41]:
# The directory where the data splits will be saved.
split_path = os.path.join('.', 'Dissertation_Data_2')
# Split labeled data into train, valid, and test csv files to disk, with the split ratio of 3:1:1.
dm.data.split(C, split_path, 'train.csv', 'valid.csv', 'test.csv',
              [3, 1, 1])

In [ ]:
# Load the training data files from the disk. Ignore the "left_id" and "right_id" 
# columns for data preprocessing.
# The 'use_magellan_convention' parameter asks deepmatcher to use Magellan's 
# naming convention for the left and right table column name prefixes 
# ("ltable_", and "rtable_"), and also to consider "_id" as the ID column.
train, validation, test = dm.data.process(
    path=os.path.join('.', 'Dissertation_Data_2'),
    cache='train_cache.pth',
    train='train.csv',
    validation='valid.csv',
    test='test.csv',
    use_magellan_convention=True,
    ignore_columns=('ltable_id', 'rtable_id'))


Reading and processing data from "./Dissertation_Data_2/train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "./Dissertation_Data_2/test.csv"
0% [############################# ] 100% | ETA: 00:00:00INFO:deepmatcher.data.field:Downloading vectors from https://drive.google.com/uc?export=download&id=1Vih8gAmgBnuYDxfblbT94P6WjB7s1ZSh to /root/.vector_cache/wiki.en.bin
INFO:deepmatcher.data.field:Unable to fetch cached English Word Embeddings from https://drive.google.com/uc?export=download&id=1Vih8gAmgBnuYDxfblbT94P6WjB7s1ZSh


In [ ]:
# Create a hybrid model.
model = dm.MatchingModel(attr_summarizer='sif', attr_comparator='diff')
model.initialize(train)  # Explicitly initialize model.

In [ ]:
# Train the hybrid model with 10 training epochs, batch size of 16, positive-to-negative 
# ratio to be 3. We save the best model (with the 
# highest F1 score on the validation set) to 'hybrid_model.pth'.
model.run_train(
    train,
    validation,
    epochs=10,
    batch_size=16,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)